In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("C:/Users/input"))

# Any results you write to the current directory are saved as output.

In [2]:
# lets import some stuff
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical
import re
from keras import regularizers
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('C:/Users/HP/Downloads/train.tsv', delimiter='\t')
df = df[['Phrase', 'Sentiment']]

pd.set_option('display.max_colwidth', -1)
df.head(3)

C:\Users\HP\AppData\Local\Temp\ipykernel_11216\3657699696.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Phrase,Sentiment
0,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,A series of escapades demonstrating the adage that what is good for the goose,2
2,A series,2


In [4]:
df['Sentiment'].unique()

array([1, 2, 3, 4, 0], dtype=int64)

In [4]:
def clean_str(in_str):
    in_str = str(in_str)
    # replace urls with 'url'
    in_str = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", in_str)
    in_str = re.sub(r'([^\s\w]|_)+', '', in_str)
    return in_str.strip().lower()


df['text'] = df['Phrase'].apply(clean_str)

In [5]:
df.Sentiment.value_counts()

2    79582
3    32927
1    27273
4    9206 
0    7072 
Name: Sentiment, dtype: int64

In [6]:
df_0 = df[df['Sentiment'] == 0].sample(frac=1)
df_1 = df[df['Sentiment'] == 1].sample(frac=1)
df_2 = df[df['Sentiment'] == 2].sample(frac=1)
df_3 = df[df['Sentiment'] == 3].sample(frac=1)
df_4 = df[df['Sentiment'] == 4].sample(frac=1)

# we want a balanced set for training against - there are 7072 `0` examples
sample_size = 7072

data = pd.concat([df_0.head(sample_size), df_1.head(sample_size), df_2.head(sample_size), df_3.head(sample_size), df_4.head(sample_size)]).sample(frac=1)

In [7]:
data['l'] = data['Phrase'].apply(lambda x: len(str(x).split(' ')))
print("mean length of sentence: " + str(data.l.mean()))
print("max length of sentence: " + str(data.l.max()))
print("std dev length of sentence: " + str(data.l.std()))

mean length of sentence: 9.109756787330317
max length of sentence: 52
std dev length of sentence: 8.006182447173451


In [8]:
# these sentences aren't that long so we may as well use the whole string
sequence_length = 52

In [9]:
sequence_length = 52
max_features = 20000 # this is the number of words we care about

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(data['Phrase'].values)

# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(data['Phrase'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)

y = pd.get_dummies(data['Sentiment']).values

# where there isn't a test set, Kim keeps back 10% of the data for testing, I'm going to do the same since we have an ok amount to play with
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size " + str(len(X_test)))

test set size 3536


In [ ]:
#LR

In [12]:
from sklearn.model_selection import train_test_split
review = data['Phrase'].values
label = data['Sentiment'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
review_vectorizer = CountVectorizer()
review_vectorizer.fit(review_train)
Xlr_train = review_vectorizer.transform(review_train)
Xlr_test  = review_vectorizer.transform(review_test)
Xlr_train

<26520x13323 sparse matrix of type '<class 'numpy.int64'>'
	with 205253 stored elements in Compressed Sparse Row format>

In [15]:
from sklearn.linear_model import LogisticRegression
LRmodel = LogisticRegression()
LRmodel.fit(Xlr_train, label_train)
score = LRmodel.score(Xlr_test, label_test)
print("Accuracy:", score) 

Accuracy: 0.5432126696832579


C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#NB

In [16]:
from sklearn.naive_bayes import MultinomialNB
nb_tfidf = MultinomialNB()
nb_tfidf.fit(Xlr_train, label_train)  #model

MultinomialNB()

In [19]:
y_predict = nb_tfidf.predict(Xlr_test)
#y_prob = nb_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]
 

print(classification_report(label_test,y_predict))
#print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

              precision    recall  f1-score   support

           0       0.50      0.70      0.58      1761
           1       0.41      0.38      0.40      1764
           2       0.50      0.22      0.31      1765
           3       0.39      0.33      0.36      1752
           4       0.55      0.75      0.63      1798

    accuracy                           0.48      8840
   macro avg       0.47      0.48      0.46      8840
weighted avg       0.47      0.48      0.46      8840



In [ ]:
#Model 1: Random embeddings

In [28]:
embedding_dim = 300 # Kim uses 300 here
num_filters = 100

inputs = Input(shape=(sequence_length,), dtype='int32')

# use a random embedding for the text
embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=sequence_length)(inputs)

reshape = Reshape((sequence_length, embedding_dim, 1))(embedding_layer)

# Note the relu activation which Kim specifically mentions
# He also uses an l2 constraint of 3
# Also, note that the convolution window acts on the whole 200 dimensions - that's important
conv_0 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)

# perform max pooling on each of the convoluations
maxpool_0 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2)

# concat and flatten
concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)

# do dropout and predict
dropout = Dropout(0.5)(flatten)
output = Dense(units=5, activation='softmax')(dropout)

In [29]:
model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 52, 300)      6000000     ['input_3[0][0]']                
                                                                                                  
 reshape_2 (Reshape)            (None, 52, 300, 1)   0           ['embedding_2[0][0]']            
                                                                                                  
 conv2d_3 (Conv2D)              (None, 50, 1, 100)   90100       ['reshape_2[0][0]']              
                                                                                            

In [30]:
batch_size = 32 # Kim uses 50 here, I have a slightly smaller sample size than num
history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=1, validation_split=0.1, shuffle=True)

Epoch 1/10
896/896 [==============================] - 80s 89ms/step - loss: 1.6991 - accuracy: 0.2386 - val_loss: 1.6376 - val_accuracy: 0.2806
Epoch 2/10
896/896 [==============================] - 95s 106ms/step - loss: 1.6188 - accuracy: 0.2893 - val_loss: 1.6294 - val_accuracy: 0.3305
Epoch 3/10
896/896 [==============================] - 81s 91ms/step - loss: 1.5862 - accuracy: 0.3261 - val_loss: 1.6816 - val_accuracy: 0.3277
Epoch 4/10
896/896 [==============================] - 79s 88ms/step - loss: 1.5510 - accuracy: 0.3543 - val_loss: 1.5516 - val_accuracy: 0.3704
Epoch 5/10
896/896 [==============================] - 79s 88ms/step - loss: 1.5190 - accuracy: 0.3744 - val_loss: 1.5256 - val_accuracy: 0.3789
Epoch 6/10
896/896 [==============================] - 80s 89ms/step - loss: 1.4941 - accuracy: 0.3881 - val_loss: 1.5327 - val_accuracy: 0.3541
Epoch 7/10
896/896 [==============================] - 79s 89ms/step - loss: 1.4777 - accuracy: 0.4017 - val_loss: 1.5520 - val_accuracy

In [20]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

KeyError: 'acc'

In [31]:
y_hat = model.predict(X_test)
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))

111/111 [==============================] - 1s 8ms/step


array([[118, 162,  94, 159, 148],
       [ 55, 186, 299, 110,  42],
       [ 18, 110, 528,  41,  19],
       [ 71, 163, 238, 112, 127],
       [ 89, 104,  59, 110, 374]], dtype=int64)

In [32]:
#Model 2: Static word2vec

In [33]:
embeddings_index = {}
f = open(os.path.join('C:/Users/HP/Downloads/', 'glove.42B.300d.txt'),encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [34]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 14114 unique tokens.


In [35]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

14115


In [36]:
inputs_2 = Input(shape=(sequence_length,), dtype='int32')

# note the `trainable=False`, later we will make this layer trainable
embedding_layer_2 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=False)(inputs_2)

reshape_2 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_2)

conv_0_2 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_1_2 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_2_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)

maxpool_0_2 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_2)
maxpool_1_2 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_2)
maxpool_2_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_2)

concatenated_tensor_2 = Concatenate(axis=1)([maxpool_0_2, maxpool_1_2, maxpool_2_2])
flatten_2 = Flatten()(concatenated_tensor_2)

dropout_2 = Dropout(0.5)(flatten_2)
output_2 = Dense(units=5, activation='softmax')(dropout_2)

In [37]:
model_2 = Model(inputs=inputs_2, outputs=output_2)
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 52, 300)      4234500     ['input_4[0][0]']                
                                                                                                  
 reshape_3 (Reshape)            (None, 52, 300, 1)   0           ['embedding_3[0][0]']            
                                                                                                  
 conv2d_6 (Conv2D)              (None, 50, 1, 100)   90100       ['reshape_3[0][0]']              
                                                                                            

In [38]:
batch_size = 32
history_2 = model_2.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=1, validation_split=0.2)

Epoch 1/10
796/796 [==============================] - 26s 31ms/step - loss: 1.8465 - accuracy: 0.2768 - val_loss: 1.7531 - val_accuracy: 0.2812
Epoch 2/10
796/796 [==============================] - 25s 32ms/step - loss: 1.7494 - accuracy: 0.2835 - val_loss: 1.7192 - val_accuracy: 0.3007
Epoch 3/10
796/796 [==============================] - 25s 32ms/step - loss: 1.7233 - accuracy: 0.2865 - val_loss: 1.7073 - val_accuracy: 0.3271
Epoch 4/10
796/796 [==============================] - 25s 32ms/step - loss: 1.7044 - accuracy: 0.2972 - val_loss: 1.6898 - val_accuracy: 0.3904
Epoch 5/10
796/796 [==============================] - 26s 33ms/step - loss: 1.6923 - accuracy: 0.2990 - val_loss: 1.6601 - val_accuracy: 0.3626
Epoch 6/10
796/796 [==============================] - 25s 32ms/step - loss: 1.6677 - accuracy: 0.3085 - val_loss: 1.6661 - val_accuracy: 0.3128
Epoch 7/10
796/796 [==============================] - 26s 32ms/step - loss: 1.6593 - accuracy: 0.3208 - val_loss: 1.6482 - val_accuracy:

In [ ]:
plt.plot(history_2.history['acc'])
plt.plot(history_2.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [39]:

y_hat_2 = model_2.predict(X_test)
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))

111/111 [==============================] - 1s 9ms/step


array([[562,  28,  14,   5,  72],
       [406,  75,  80,  11, 120],
       [171,  45, 297,  38, 165],
       [161,  27, 126,  36, 361],
       [112,   6,  28,  16, 574]], dtype=int64)

In [40]:
#Model 3: w2v with trainable embeddings

In [41]:
inputs_3 = Input(shape=(sequence_length,), dtype='int32')
embedding_layer_3 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_3)

reshape_3 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_3)

# note the relu activation
conv_0_3 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_1_3 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_2_3 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)

maxpool_0_3 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_3)
maxpool_1_3 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_3)
maxpool_2_3 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_3)

concatenated_tensor_3 = Concatenate(axis=1)([maxpool_0_3, maxpool_1_3, maxpool_2_3])
flatten_3 = Flatten()(concatenated_tensor_3)

dropout_3 = Dropout(0.5)(flatten_3)
output_3 = Dense(units=5, activation='softmax')(dropout_3)

In [42]:
model_3 = Model(inputs=inputs_3, outputs=output_3)
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_3.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 52, 300)      4234500     ['input_5[0][0]']                
                                                                                                  
 reshape_4 (Reshape)            (None, 52, 300, 1)   0           ['embedding_4[0][0]']            
                                                                                                  
 conv2d_9 (Conv2D)              (None, 50, 1, 100)   90100       ['reshape_4[0][0]']              
                                                                                            

In [43]:
batch_size = 32
history_3 = model_3.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=1, validation_split=0.2)

Epoch 1/10
796/796 [==============================] - 63s 78ms/step - loss: 1.8980 - accuracy: 0.3217 - val_loss: 1.8569 - val_accuracy: 0.4740
Epoch 2/10
796/796 [==============================] - 62s 78ms/step - loss: 1.8128 - accuracy: 0.4589 - val_loss: 1.8101 - val_accuracy: 0.4099
Epoch 3/10
796/796 [==============================] - 62s 78ms/step - loss: 1.7160 - accuracy: 0.5052 - val_loss: 1.7186 - val_accuracy: 0.5159
Epoch 4/10
796/796 [==============================] - 62s 78ms/step - loss: 1.6611 - accuracy: 0.5312 - val_loss: 1.6614 - val_accuracy: 0.5218
Epoch 5/10
796/796 [==============================] - 62s 78ms/step - loss: 1.6209 - accuracy: 0.5379 - val_loss: 1.6140 - val_accuracy: 0.5350
Epoch 6/10
796/796 [==============================] - 62s 78ms/step - loss: 1.5658 - accuracy: 0.5599 - val_loss: 1.6051 - val_accuracy: 0.5181
Epoch 7/10
796/796 [==============================] - 62s 78ms/step - loss: 1.5661 - accuracy: 0.5599 - val_loss: 1.5867 - val_accuracy:

In [ ]:
plt.plot(history_3.history['acc'])
plt.plot(history_3.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_3.history['loss'])
plt.plot(history_3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [44]:
y_hat_3 = model_3.predict(X_test)
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))

111/111 [==============================] - 1s 8ms/step


array([[267, 315,  68,  31,   0],
       [ 87, 297, 239,  62,   7],
       [ 13,  85, 504, 106,   8],
       [  5,  32, 196, 346, 132],
       [  6,  13,  26, 335, 356]], dtype=int64)

In [45]:
#Model 4: Again, but with binary classification (similar to SST2)

In [46]:
# SST2 is the same as SST1, but with neutrals removed and binary labels
sst2_data = pd.concat([df_0.head(sample_size), df_1.head(sample_size), df_3.head(sample_size), df_4.head(sample_size)]).sample(frac=1)

def merge_sentiments(x):
    if x == 0 or x == 1:
        return 0
    else:
        return 1

sst2_data['Sentiment'] = sst2_data['Sentiment'].apply(merge_sentiments)

sst2_tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
sst2_tokenizer.fit_on_texts(sst2_data['Phrase'].values)

sst2_X = sst2_tokenizer.texts_to_sequences(sst2_data['Phrase'].values)
sst2_X = pad_sequences(sst2_X, sequence_length)

sst2_y = sst2_data['Sentiment'].values

sst2_X_train, sst2_X_test, sst2_y_train, sst2_y_test = train_test_split(sst2_X, sst2_y, test_size=0.1)

In [47]:
inputs_4 = Input(shape=(sequence_length,), dtype='int32')
embedding_layer_4 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_4)

reshape_4 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_4)

conv_0_4 = Conv2D(num_filters, kernel_size=(3, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
conv_1_4 = Conv2D(num_filters, kernel_size=(4, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
conv_2_4 = Conv2D(num_filters, kernel_size=(5, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)

maxpool_0_4 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_4)
maxpool_1_4 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_4)
maxpool_2_4 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_4)

concatenated_tensor_4 = Concatenate(axis=1)([maxpool_0_4, maxpool_1_4, maxpool_2_4])
flatten_4 = Flatten()(concatenated_tensor_4)

dropout_4 = Dropout(0.5)(flatten_4)
# note the different activation
output_4 = Dense(units=1, activation='sigmoid')(dropout_4)

In [48]:
model_4 = Model(inputs=inputs_4, outputs=output_4)

# note we're using binary_crossentropy here instead of categorical
model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_4.summary())

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 52, 300)      4234500     ['input_6[0][0]']                
                                                                                                  
 reshape_5 (Reshape)            (None, 52, 300, 1)   0           ['embedding_5[0][0]']            
                                                                                                  
 conv2d_12 (Conv2D)             (None, 50, 1, 100)   90100       ['reshape_5[0][0]']              
                                                                                            

In [49]:
batch_size = 32
history_4 = model_4.fit(sst2_X_train, sst2_y_train, epochs=10, batch_size=batch_size, verbose=1, validation_split=0.2)

Epoch 1/10
637/637 [==============================] - 74s 115ms/step - loss: 130.5147 - accuracy: 0.5086 - val_loss: 0.6971 - val_accuracy: 0.4980
Epoch 2/10
637/637 [==============================] - 81s 127ms/step - loss: 0.6987 - accuracy: 0.5030 - val_loss: 0.6973 - val_accuracy: 0.5024
Epoch 3/10
637/637 [==============================] - 81s 127ms/step - loss: 0.6979 - accuracy: 0.5018 - val_loss: 0.6973 - val_accuracy: 0.4976
Epoch 4/10
637/637 [==============================] - 81s 126ms/step - loss: 0.6969 - accuracy: 0.4922 - val_loss: 0.6956 - val_accuracy: 0.5031
Epoch 5/10
637/637 [==============================] - 80s 126ms/step - loss: 0.6956 - accuracy: 0.4991 - val_loss: 0.6949 - val_accuracy: 0.5024
Epoch 6/10
637/637 [==============================] - 80s 125ms/step - loss: 0.6946 - accuracy: 0.5021 - val_loss: 0.6941 - val_accuracy: 0.5024
Epoch 7/10
637/637 [==============================] - 80s 126ms/step - loss: 0.6940 - accuracy: 0.4971 - val_loss: 0.6938 - val_

In [ ]:
plt.plot(history_4.history['acc'])
plt.plot(history_4.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_4.history['loss'])
plt.plot(history_4.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [50]:
y_hat_4 = model_4.predict(sst2_X_test)

accuracy_score(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))
confusion_matrix(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))

89/89 [==============================] - 2s 17ms/step


array([[1400,    0],
       [1429,    0]], dtype=int64)

In [51]:
print("CNN random       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))))
print("CNN static       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))))
print("CNN trainable    : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))))
print("Binary trainable : " + str(accuracy_score(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))))

CNN random       : 0.372737556561086
CNN static       : 0.43665158371040724
CNN trainable    : 0.5005656108597285
Binary trainable : 0.4948745139625309
